#### IMPORT PACKAGES

In [23]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI
import pyodbc

#### CONNECT TO AZURE SQL DB

In [24]:
# odbc connection string
server = '<server>'
database = '<database>'
username = '<username>'
password = '<password>'
driver = 'ODBC Driver 18 for SQL Server'

In [25]:
# connect python to azure sql db
connectionString = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver={driver}"

try:
    db = SQLDatabase.from_uri(connectionString)

except pyodbc.Error as e:
    # Handle any errors that occur during the connection
    print(f"Error connecting to Azure SQL Database: {str(e)}")

#### SET OPENAI API KEY

In [26]:
# set API keys to authenticate requests to the API
OPENAI_API_KEY = "<API Key"

#### SET UP AGENT

In [27]:
# create new llm model
llm = ChatOpenAI(
    temperature=0, 
    openai_api_key=OPENAI_API_KEY, 
    model_name='gpt-3.5-turbo')

# toolkit for interacting with sql databases
toolkit = SQLDatabaseToolkit(
    db=db,
    llm=llm)

# initialize agent
agent_executor = create_sql_agent(
     llm=llm,
     toolkit=toolkit,
     verbose=True,
     agent_type=AgentType.OPENAI_FUNCTIONS # what does this do?
    )

#### RUN QUERIES

In [28]:
# query 1
agent_executor.run("how many rows are there?")



> Entering new AgentExecutor chain...



Invoking: `sql_db_list_tables` with `{}`


ADMISSIONS, BuildVersion, ErrorLog
Invoking: `sql_db_query` with `SELECT COUNT(*) FROM ADMISSIONS`


[(129,)]There are 129 rows in the ADMISSIONS table.

> Finished chain.


'There are 129 rows in the ADMISSIONS table.'

In [30]:
# query 2
agent_executor.run("What is the frequency of ethnicities?")



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with `{}`


ADMISSIONS, BuildVersion, ErrorLog
Invoking: `sql_db_schema` with `ADMISSIONS`



CREATE TABLE [ADMISSIONS] (
	row_id INTEGER NOT NULL, 
	subject_id INTEGER NOT NULL, 
	hadm_id INTEGER NOT NULL, 
	admittime DATETIME2 NOT NULL, 
	dischtime DATETIME2 NOT NULL, 
	deathtime DATETIME2 NULL, 
	admission_type NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	admission_location NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	discharge_location NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	insurance NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	language NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	religion NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	marital_status NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	ethnicity NVARCHAR(100) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	edregtime DATETIME2 NULL, 
	edouttim

'The frequency of ethnicities in the database is as follows:\n\n- WHITE: 86\n- HISPANIC/LATINO - PUERTO RICAN: 15\n- UNKNOWN/NOT SPECIFIED: 11\n- BLACK/AFRICAN AMERICAN: 7\n- OTHER: 3\n- HISPANIC OR LATINO: 2\n- ASIAN: 2\n- AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE: 2\n- UNABLE TO OBTAIN: 1'